In [ ]:
#Final code - Downloading and deleting the h5 file after use, skipping missing day data and adding date in the dataframe - 3 august
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import xarray as xr
from io import BytesIO
import requests
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.request
import h5py
import os
import pandas as pd
from selenium.common.exceptions import TimeoutException
def wait_for_downloads(download_path):
    while any(filename.endswith(".crdownload") for filename in os.listdir(download_path)):
        print("Waiting for downloads to complete...")
        time.sleep(5)
def get_chrome_options(download_path):
    options = Options()
    options.add_experimental_option('prefs', {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
    })
    return options
#download_path = "F:\\soil_moisture\\may30"
def download_h5_files(url, download_path,date_str1):
    # Initialize Chrome WebDriver with download options
    chrome_options = get_chrome_options(download_path)
    driver = webdriver.Chrome(executable_path="C:\\Users\\Acer\\Desktop\\chrome driver\\chromedriver-win64\\chromedriver.exe", options=chrome_options)
    driver.maximize_window()
    driver.get(url)

    # Find the username and password input elements and fill them with your credentials
    username_input = driver.find_element_by_name("username")  # Replace "username" with the actual name of the username input field
    password_input = driver.find_element_by_name("password")  # Replace "password" with the actual name of the password input field
    username = "toliresearch"  # Enter your username
    password = "Researchgarnetoli4#"  # Enter your password
    username_input.send_keys(username)
    password_input.send_keys(password)

    # Submit the login form
    login_button = driver.find_element(By.NAME, "commit")  # Replace 'login_button_id' with the actual ID of the login button element
    login_button.click()
    
    # Wait for the visibility of the links
    wait = WebDriverWait(driver, 20)
    #element = wait.until(EC.visibility_of_element_located((By.LINK_TEXT, "2015.04.11/")))
    try:
        element = wait.until(EC.visibility_of_element_located((By.LINK_TEXT, date_str1)))
        element.click()

    # ... Rest of the code for downloading the .h5 files ...
    except TimeoutException:
        print(f"Data not available for {date_str1}. Skipping to the next date.")
        driver.quit()
        return  # Skip the rest of the function for this date
    
    #element = wait.until(EC.visibility_of_element_located((By.LINK_TEXT, date_str1)))
    #element.click()

    # Get all the links to .h5 files
    links = driver.find_elements(By.XPATH, "//a[contains(@href, '.h5')]")

    # Create an empty set to keep track of downloaded file names
    downloaded_files = set()

# Loop through the links and download the files
    for link in links:
        file_url = link.get_attribute("href")
        file_name = os.path.basename(file_url)
             
        
        # Check if the file has the .h5 extension, if not, skip to the next link
        if not file_name.endswith(".h5"):
            continue

        file_path = os.path.join(download_path, file_name)
        
        # Check if the file has already been downloaded, if yes, skip to the next date
        if os.path.exists(file_path):
            print(f"Skipping download for {file_name} as it already exists.")
            continue
        elif file_name in downloaded_files:
            continue
        else:
            try:
                print(f"Downloading {file_name}")
                link.click()
                #urllib.request.urlretrieve(file_url, file_path)
                # Add a delay to avoid overloading the server
                #wait.until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Download complete')]")))
                #print(f"{file_name} download complete")
                time.sleep(10)
                downloaded_files.add(file_name)
            except Exception as e:
                print(f"Failed to download {file_name}: {e}")
    wait_for_downloads(download_path)
    driver.quit()
    #time.sleep(120)
   # Replace 'folder_path' with the path to the folder containing the .h5 files
    folder_path = download_path #"F:\\soil_moisture\\june11"

    # Create an empty list to store DataFrames from each file
    dfs = []

    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.h5'):
            file_path = os.path.join(folder_path, filename)

            with h5py.File(file_path, 'r') as h5_file:
                dataset_path = "Soil_Moisture_Retrieval_Data/longitude_centroid"
                dataset_path1 = "Soil_Moisture_Retrieval_Data/latitude_centroid"
                dataset_path2 = "Soil_Moisture_Retrieval_Data/soil_moisture"
                dataset_path3 = "Soil_Moisture_Retrieval_Data/soil_moisture_error"
                dataset_path4 = "Soil_Moisture_Retrieval_Data/soil_moisture_option1"
                dataset_path5 = "Soil_Moisture_Retrieval_Data/soil_moisture_option2"
                dataset_path6 = "Soil_Moisture_Retrieval_Data/latitude"
                dataset_path7 = "Soil_Moisture_Retrieval_Data/longitude"

                # Check if the datasets exist in the file
                if dataset_path in h5_file and dataset_path1 in h5_file and dataset_path2 in h5_file:
                    # Access the datasets using their paths
                    dataset = h5_file[dataset_path]
                    dataset1 = h5_file[dataset_path1]
                    dataset2 = h5_file[dataset_path2]
                    dataset3 = h5_file[dataset_path3]
                    dataset4 = h5_file[dataset_path4]
                    dataset5 = h5_file[dataset_path5]
                    dataset6 = h5_file[dataset_path6]
                    dataset7 = h5_file[dataset_path7]
                    
                    # Read the data into NumPy arrays
                    data = dataset[()]
                    data1 = dataset1[()]
                    data2 = dataset2[()]
                    data3 = dataset3[()]
                    data4 = dataset4[()]
                    data5 = dataset5[()]
                    data6 = dataset6[()]
                    data7 = dataset7[()]
                    

                    # Create a DataFrame from the data arrays
                    df = pd.DataFrame({
                        'longitude_centroid': data,
                        'latitude_centroid': data1,
                        'soil moisture': data2,
                        'soil_moisture_error': data3,
                        'soil_moisture_option1': data4,
                        'soil_moisture_option2': data5,
                        'latitude': data6,
                        'longitude': data7
                    })

                    # Append the DataFrame to the list
                    dfs.append(df)
                else:
                    print(f"One or more datasets not found in the HDF5 file: {file_path}")
    file_path=download_path+"\\"+"output1.xlsx"
    # Combine all DataFrames into a single DataFrame (row-bind them)
    if dfs:
        final_df = pd.concat(dfs, ignore_index=True)
        #print(final_df)
    else:
        print("No valid HDF5 files found in the folder.")
    
    df.to_excel(file_path, index=False)  # Set index=False to exclude row numbers from the output

    #print("DataFrame has been successfully written to Excel file:", file_path)
    min_longitude = 78
    max_longitude = 90
    min_latitude = 25
    max_latitude = 35

    # Create boolean masks for filtering
    longitude_mask = (df['longitude_centroid'] >= min_longitude) & (df['longitude_centroid'] <= max_longitude)
    latitude_mask = (df['latitude_centroid'] >= min_latitude) & (df['latitude_centroid'] <= max_latitude)

    # Use the masks to filter the DataFrame
    filtered_df = df[longitude_mask & latitude_mask]
    file_path1=download_path+"\\"+"Nepal_output.xlsx"
    if len(filtered_df):
        single_date=date_str1[:-1]
        single_date1 = single_date.replace(".", "-")
        single_date1 = pd.to_datetime(single_date1)
        single_date1 = single_date1.strftime('%Y-%m-%d')
        # Use .loc to update the 'Date' column
        filtered_df.loc[:, 'Date'] = single_date1
    else:
        filtered_df['Date']=None
        #filtered_df['Date'] = single_date1
        # Write the DataFrame to an Excel file
    filtered_df .to_excel(file_path1, index=False)  # Set index=False to exclude row numbers from the output

    #print("DataFrame has been successfully written to Excel file:", file_path)
    #print(filtered_df)

    #driver.quit()

if __name__ == "__main__":
    base_download_path = "C:\\Users\\Acer\\Desktop\\Soil_moisture_SMAP\\all_day"  # Replace this with the base download location

    start_date = datetime(2021, 11, 20)
    end_date = datetime(2023, 8, 12)

    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime("%Y.%m.%d")
        folder_name = current_date.strftime("%Y.%m.%d") # current_date.strftime("%B%d").lower()  # Create a folder name like "april10"
        download_path = os.path.join(base_download_path, folder_name)

        # Create the folder if it doesn't exist
        if not os.path.exists(download_path):
            os.makedirs(download_path)

        #full_url = "https://n5eil01u.ecs.nsidc.org/SMAP/SPL2SMP.008/" + date_str + "/"
        date_str1=date_str + "/"
        full_url = "https://n5eil01u.ecs.nsidc.org/SMAP/SPL2SMP.008/"
        download_h5_files(full_url, download_path,date_str1)
        # Code added for deleting the downloaded .h5 file
        for filename in os.listdir(download_path):
            if filename.endswith('.h5'):
                file_path = os.path.join(download_path, filename)
                os.remove(file_path)

        # Move to the next date
        current_date += timedelta(days=1)

Skipping download for SMAP_L2_SM_P_36335_D_20211120T002559_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36336_A_20211120T011510_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36336_D_20211120T020425_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36337_A_20211120T025340_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36337_D_20211120T034255_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36338_A_20211120T043205_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36338_D_20211120T052120_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36339_A_20211120T061035_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36339_D_20211120T065950_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36340_A_20211120T074901_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36340_D_2021112

Skipping download for SMAP_L2_SM_P_36361_D_20211121T190554_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36362_A_20211121T195509_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36362_D_20211121T204424_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36363_A_20211121T213339_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36363_D_20211121T222250_R18290_001.h5 as it already exists.
Skipping download for SMAP_L2_SM_P_36364_A_20211121T231204_R18290_001.h5 as it already exists.
